<a href="https://colab.research.google.com/github/MaderG/idw-parquet-orc/blob/main/Parquet_e_ORC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demonstração Prática: Formatos Colunares Parquet e ORC**

Iremos aprofundar um pouco mais sobre os formatos colunares (Parquet e ORC) comparados com formatos lineares (CSV), destacando as vantagens em termos de:
- Tamanho de armazenamento
- Velocidade de leitura e escrita
- Eficiência em consultas analíticas
- Técnicas de otimização

## 1. Instalação das Bibliotecas Necessárias

Primeiro, vamos instalar as bibliotecas Python necessárias para trabalhar com Parquet e ORC.

In [ ]:
!pip install pandas numpy pyarrow matplotlib seaborn --quiet

## 2. Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.orc as orc
import time
import os

# Configurações para melhor visualização
pd.set_option('display.max_columns', None)

## 3. Operações Básicas em Parquet

### Escrita

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# escrita

df = pd.DataFrame({
    'nome': ['Ana', 'José', 'Carla'],
    'matricula': [987, 654, 321],
    'curso': ['Matemática', 'Física', 'Química']
})

tabela = pa.Table.from_pandas(df)
pq.write_table(tabela, 'alunos.parquet')
print("Dados gravados em alunos.parquet")



Dados gravados em alunos.parquet


1. Importamos as bibliotecas pyarrow e pyarrow.parquet.
2. Criamos um DataFrame do pandas com os dados dos estudantes.
3. Convertemos o DataFrame para uma tabela do PyArrow usando `pa.Table.from_pandas()`.
4. Usamos a função `pq.write_table()` para escrever a tabela em um arquivo Parquet chamado
estudantes.parquet.


### Leitura

In [ ]:
tabela_leitura = pq.read_table('alunos.parquet')
df_leitura = tabela_leitura.to_pandas()
print(df_leitura)

    nome  matricula       curso
0    Ana        987  Matemática
1   José        654      Física
2  Carla        321     Química


1. Para a leitura, usamos a função ```pq.read_table()``` para ler o arquivo
2. Convertemos a tabela do PyArrow para o DataFrame do pandas usando tabela_leitura.to_pandas()`
3. Imprimimos



## 4. Operações Básicas em ORC

Para realizar as operações em ORC trocaremos, somente o pyarrow.pq, por pyarrow.orc, e repetiremos os mesmos processos

### Leitura

In [ ]:
import pyarrow as pa
import pyarrow.orc as orc
import pandas as pd
# Criando um DataFrame do pandas
df = pd.DataFrame({
    'nome': ['Ana', 'José', 'Carla'],
    'matricula': [987, 654, 321],
    'curso': ['Matemática', 'Física', 'Química']
})
# Convertendo o DataFrame para uma tabela do PyArrow
tabela = pa.Table.from_pandas(df)
# Escrevendo a tabela em um arquivo ORC
orc.write_table(tabela, 'estudantes.orc')
print("Dados gravados em estudantes.orc")


Dados gravados em estudantes.orc


### Escrita

In [ ]:
tabela_leitura = orc.read_table('estudantes.orc', columns=['nome', 'matricula'])
df_leitura = tabela_leitura.to_pandas()
print(df_leitura)


    nome  matricula
0    Ana        987
1   José        654
2  Carla        321


## 5. Criação de Dados de Exemplo

Vamos criar um conjunto de dados simulando registros de vendas de uma empresa varejista.

In [ ]:
def criar_dados_exemplo(n_registros=5_000_000):
    np.random.seed(42)

    df = pd.DataFrame({
        'data': pd.date_range(start='2023-01-01', periods=n_registros, freq='30s'),
        'id_produto': np.random.randint(1, 1000, n_registros),
        'categoria': np.random.choice(['Eletrônicos', 'Roupas', 'Alimentos', 'Casa', 'Jardim'], n_registros),
        'região': np.random.choice(['Norte', 'Sul', 'Leste', 'Oeste', 'Centro'], n_registros),
        'valor_venda': np.random.uniform(10, 1000, n_registros).round(2),
        'quantidade': np.random.randint(1, 10, n_registros),
        'desconto': np.random.uniform(0, 0.3, n_registros).round(2),
        'cliente_id': np.random.randint(1, 10000, n_registros),
        'avaliação': np.random.randint(1, 6, n_registros)
    })

    # Calculando o valor total (com desconto aplicado)
    df['valor_total'] = (df['valor_venda'] * df['quantidade'] * (1 - df['desconto'])).round(2)

    return df

df_vendas = criar_dados_exemplo()
df_vendas.head()

,data,id_produto,categoria,região,valor_venda,quantidade,desconto,cliente_id,avaliação,valor_total
0,2023-01-01 00:00:00,103,Roupas,Centro,724.46,1,0.08,2754,1,666.50
1,2023-01-01 00:00:30,436,Eletrônicos,Centro,549.63,4,0.13,4605,1,1912.71
2,2023-01-01 00:01:00,861,Eletrônicos,Centro,448.62,4,0.18,2273,3,1471.47
3,2023-01-01 00:01:30,271,Casa,Norte,677.69,4,0.10,4406,5,2439.68
4,2023-01-01 00:02:00,107,Eletrônicos,Leste,413.42,4,0.15,341,5,1405.63


## 6. Comparação de Formatos: CSV vs Parquet vs ORC

Vamos comparar os três formatos em termos de tamanho de armazenamento e tempo de escrita.

In [ ]:
def medir_operacao(operacao, *args, **kwargs):
        inicio = time.time()
        resultado = operacao(*args, **kwargs)
        tempo = time.time() - inicio
        return resultado, tempo

### Salvando os dados em CSV

In [ ]:
_, tempo_csv = medir_operacao(df_vendas.to_csv, 'vendas.csv', index=False)
tamanho_csv = os.path.getsize('vendas.csv') / (1024 * 1024)  # MB

### Salvando os dados em Parquet

In [ ]:
metodos_compressao = ['snappy', 'gzip', 'brotli', None]
resultados_parquet = []

for compressao in metodos_compressao:
    nome_arquivo = f'vendas_{compressao if compressao else "sem"}.parquet'
    _, tempo = medir_operacao(df_vendas.to_parquet, nome_arquivo, index=False, compression=compressao)
    tamanho = os.path.getsize(nome_arquivo) / (1024 * 1024)  # MB
    resultados_parquet.append({
        'formato': f'Parquet ({compressao if compressao else "sem compressão"})',
        'tempo': tempo,
        'tamanho': tamanho
    })

### Salvando em ORC

In [ ]:
tabela = pa.Table.from_pandas(df_vendas)
_, tempo_orc = medir_operacao(lambda: orc.write_table(tabela, 'vendas.orc'))
tamanho_orc = os.path.getsize('vendas.orc') / (1024 * 1024)  # MB


### Comparação entre os resultados

In [ ]:
resultados_formatos = [
    {'formato': 'CSV', 'tempo': tempo_csv, 'tamanho': tamanho_csv},
    *resultados_parquet,
    {'formato': 'ORC', 'tempo': tempo_orc, 'tamanho': tamanho_orc}
]

df_resultados = pd.DataFrame(resultados_formatos)
df_resultados

,formato,tempo,tamanho
0,CSV,56.396391,315.710091
1,Parquet (snappy),3.706900,98.912908
2,Parquet (gzip),15.980401,77.658446
3,Parquet (brotli),22.070093,74.278552
4,Parquet (sem compressão),3.836204,117.854941
5,ORC,3.528821,205.957667


### **Análise dos Resultados**

**1. Tempo de Escrita**

- O CSV foi o formato mais lento para escrita, levando 56.39 segundos, enquanto o Parquet com compressão Snappy foi o mais rápido, com apenas 3.70 segundos.

- O ORC também se destacou com um tempo de escrita de 3.52 segundos, sendo uma ótima opção para cenários onde a leitura é mais frequente que a escrita.

- A compressão Brotli no Parquet, apesar de oferecer o menor tamanho de arquivo (74.27 MB), foi a mais lenta para escrita, levando 22.07 segundos.

**2. Tamanho do Arquivo**

- O CSV gerou o maior arquivo (315.71 MB), o que pode ser um problema em cenários de armazenamento em larga escala.

- O Parquet com Brotli ofereceu o menor tamanho de arquivo (74.27 MB), seguido pelo Parquet com Gzip (77.658446 MB). Isso mostra que a escolha da compressão pode impactar significativamente o espaço em disco.

- O ORC gerou um arquivo de 205.957667 MB, maior que o Parquet, mas ainda significativamente menor que o CSV.

**3. Compressão no Parquet**

- Snappy ofereceu um bom equilíbrio entre tempo de escrita e tamanho do arquivo, sendo ideal para cenários onde a velocidade é crítica.

- Gzip e Brotli são mais eficientes em termos de espaço, mas exigem mais tempo de processamento, sendo mais adequados para cenários onde o armazenamento é prioritário.

- O Parquet sem compressão ainda é uma opção viável, especialmente quando a compressão não é necessária ou quando se deseja maximizar a velocidade de escrita.

## 7. Demonstração de Leitura Seletiva de Colunas

Uma das principais vantagens dos formatos colunares é a capacidade de ler apenas as colunas necessárias.

In [ ]:
colunas_selecao = ['data', 'região', 'valor_total']

# Leitura completa do CSV
_, tempo_csv_completo = medir_operacao(pd.read_csv, 'vendas.csv')
print(f"Tempo para ler CSV completo: {tempo_csv_completo:.2f} segundos")

# Leitura seletiva do CSV (apenas algumas colunas)
_, tempo_csv_colunas = medir_operacao(pd.read_csv, 'vendas.csv', usecols=colunas_selecao)
print(f"Tempo para ler CSV (apenas 3 colunas): {tempo_csv_colunas:.2f} segundos")
print(f"Redução de tempo no CSV: {(tempo_csv_completo - tempo_csv_colunas) / tempo_csv_completo * 100:.2f}%")

# Leitura completa do Parquet
_, tempo_parquet_completo = medir_operacao(pd.read_parquet, 'vendas_snappy.parquet')
print(f"\nTempo para ler Parquet completo: {tempo_parquet_completo:.2f} segundos")

# Leitura seletiva do Parquet (apenas algumas colunas)
_, tempo_parquet_colunas = medir_operacao(pd.read_parquet, 'vendas_snappy.parquet', columns=colunas_selecao)
print(f"Tempo para ler Parquet (apenas 3 colunas): {tempo_parquet_colunas:.2f} segundos")
print(f"Redução de tempo no Parquet: {(tempo_parquet_completo - tempo_parquet_colunas) / tempo_parquet_completo * 100:.2f}%")

# Leitura completa do ORC
_, tempo_orc_completo = medir_operacao(pd.read_orc, 'vendas.orc')
print(f"\nTempo para ler ORC completo: {tempo_orc_completo:.2f} segundos")

# Leitura seletiva do ORC (apenas algumas colunas)
_, tempo_orc_colunas = medir_operacao(pd.read_orc, 'vendas.orc', columns=colunas_selecao)
print(f"Tempo para ler ORC (apenas 3 colunas): {tempo_orc_colunas:.2f} segundos")
print(f"Redução de tempo no ORC: {(tempo_orc_completo - tempo_orc_colunas) / tempo_orc_completo * 100:.2f}%")


Tempo para ler CSV completo: 11.76 segundos
Tempo para ler CSV (apenas 3 colunas): 7.27 segundos
Redução de tempo no CSV: 38.21%

Tempo para ler Parquet completo: 1.38 segundos
Tempo para ler Parquet (apenas 3 colunas): 1.04 segundos
Redução de tempo no Parquet: 24.79%

Tempo para ler ORC completo: 1.94 segundos
Tempo para ler ORC (apenas 3 colunas): 0.61 segundos
Redução de tempo no ORC: 68.54%


### Análise dos Resultados

**Leitura Completa vs. Leitura Seletiva**

- **CSV:** A leitura completa levou 11.76 segundos, enquanto a leitura seletiva (apenas 3 colunas) levou 7.27 segundos. Isso mostra que o CSV não se beneficia da leitura seletiva, pois o formato linear exige a leitura de todo o arquivo, independentemente das colunas selecionadas.

- Parquet e ORC: Ambos possuiram leituras completas rápidas e seletivas mais rápidas ainda

**Comparação entre Formatos**

- O Parquet e o ORC são muito mais eficientes que o CSV em termos de tempo de leitura, especialmente quando apenas algumas colunas são necessárias. Isso ocorre porque ambos os formatos armazenam os dados por coluna, permitindo que apenas as colunas relevantes sejam lidas do disco.

**Benefícios da Leitura Seletiva**

- Redução do tempo de leitura: Apenas as colunas necessárias são carregadas, o que é ideal para consultas analíticas.

- Economia de memória: Menos dados são carregados na memória, o que é crucial em cenários de Big Data.

- Melhor desempenho em consultas complexas: Consultas que envolvem agregações ou filtros em colunas específicas são executadas mais rapidamente.

## Conclusão

Ao longo deste tutorial, exploramos como os formatos Parquet e ORC oferecem otimização significativa para armazenamento e processamento de grandes volumes de dados. Vimos que esses formatos possibilitam uma redução no espaço ocupado, melhor desempenho em consultas e maior compatibilidade com sistemas de Big Data.

Caso seu objetivo seja trabalhar com grandes conjuntos de dados, a escolha entre Parquet e ORC dependerá de cada projeto, como leitura/escrita, compressão. Independentemente da escolha, esses formatos garantem uma melhoria considerável na eficiência do processamento de dados.